In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*
import codecs
import json
import matplotlib.pyplot as plt
import numpy as np
import nltk
import os
import requests
import statistics
import urllib
from bs4 import BeautifulSoup
import re
import sys

from urllib.request import urlopen
from multiprocessing import Pool

# Scrap Wikipedia

In [2]:
url1 = "https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Rappeur_fran%C3%A7ais"
html1 = urlopen(url1)
bs1 = BeautifulSoup(html1)
links = bs1.find("div",{"class":"mw-category"}).findAll("a" , href=re.compile("(/wiki/)+([A-Za-z0-9_:()])+"))
artists = [x['title'] for x in links]

url2 = "https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Rappeur_fran%C3%A7ais&from=S"
html2 = urlopen(url2)
bs2 = BeautifulSoup(html2)
links = bs2.find("div",{"class":"mw-category"}).findAll("a" , href=re.compile("(/wiki/)+([A-Za-z0-9_:()])+"))
artists2 = [x['title'] for x in links]

url3 ="https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Groupe_de_hip-hop_fran%C3%A7ais"
html3 = urlopen(url3)
bs3 = BeautifulSoup(html3)
links = bs3.find("div",{"class":"mw-category"}).findAll("a" , href=re.compile("(/wiki/)+([A-Za-z0-9_:()])+"))
artists3 = [x['title'] for x in links]

artists+=artists2+artists3
artists=list(set(artists))
artists = [x.split(" (")[0] for x in artists]

# Scrap Genius

In [3]:
def get_id(artist):
    requrl = '/'.join(["https://api.genius.com", "search"])
    token = "Bearer {}".format("0RpktcXrF-D2Y4Ma5ftJ8SUTcINUUBq0FksLuuFmhiNeXqDokdG9nCh2w89dRxCt")
    response = requests.get(url=requrl, params={'q':artist}, headers={"Authorization": token})
    response.raise_for_status()
    response = response.json()
    for hit in response['response']["hits"]:
        if hit["result"]["primary_artist"]["name"].lower() == artist.lower():
           artist_id = hit["result"]["primary_artist"]["id"]
           break
    return artist_id

In [4]:
def get_artist_songs_urls(artist):
    try :
        _URL_API = "https://api.genius.com/"
        ID = get_id(artist)
        page = 0
        value = 50
        URLS = []
        while value == 50:
            page += 1
            querystring = _URL_API+"artists/"+str(ID)+"/songs?per_page=50&page="+str(page)
            print(querystring)

            request = urllib.request.Request(querystring)
            request.add_header("Authorization", "Bearer " + "0RpktcXrF-D2Y4Ma5ftJ8SUTcINUUBq0FksLuuFmhiNeXqDokdG9nCh2w89dRxCt")

            response = urllib.request.urlopen(request)
            reader = codecs.getreader(response.headers.get_content_charset())
            json_obj = json.load(reader(response))

            value = len(json_obj['response']['songs'])

            for x in range(len(json_obj['response']['songs'])):
                
                url = json_obj['response']['songs'][x]['url']
                URLS.append(url)
                URLS = list(set(URLS))
        print(artist,":",len(URLS),"chansons")
    except:
        print(artist,": pas de chanson")
        URLS = False
    return URLS 

In [5]:
def get_lyrics(artist,URLS):
    lyrics = []
    long = len(URLS)
    for value,URL in enumerate(URLS):
        try:
            page = requests.get(URL)
            html = BeautifulSoup(page.text, "html.parser")
            lyric = html.find("div", class_="lyrics").get_text()
            title = html.find("title").get_text()
            title = title.split(" – ")[1]
            title = title.split(" Lyrics")[0]
            lyric = "\n\n\n" + title + " : \n" + lyric
        except:
            title = ""
            lyric = ""
        lyrics.append(lyric)
        print("\t",artist,round(100*(1+value)/long,2),"%")
    return lyrics

In [6]:
def from_artist_to_lyrics(artist):
    try:
        URLS = get_artist_songs_urls(artist)
        if URLS:
            LYRICS = get_lyrics(artist,URLS)
            with open(str(artist)+"-"+str(len(URLS))+'.txt', 'a') as file:
                file.write("".join(LYRICS))
    except:
        pass

# Multiprocessing

In [7]:
def import_datas():
    os.chdir('/home/garance/Bureau/OpenClassroom/Données textuelles/datas')
    files = [x for x in os.listdir('/home/garance/Bureau/OpenClassroom/Données textuelles/datas') if x[-4:]==".txt"]
    for file in files:
        os.remove(file)
    how_many = 50
    p = Pool(processes=how_many)
    p.map(from_artist_to_lyrics, [artist for artist in artists])
    p.close()
    
    print("FINISHED")   

In [8]:
import_datas()

Utilisateur:Leaal/Brouillon : pas de chanson
https://api.genius.com/artists/30616/songs?per_page=50&page=1
https://api.genius.com/artists/11499/songs?per_page=50&page=1
https://api.genius.com/artists/17313/songs?per_page=50&page=1
https://api.genius.com/artists/592670/songs?per_page=50&page=1
https://api.genius.com/artists/215059/songs?per_page=50&page=1
https://api.genius.com/artists/14071/songs?per_page=50&page=1
https://api.genius.com/artists/18874/songs?per_page=50&page=1
Zone d'expression populaire : pas de chanson
Smartzee : pas de chanson
Pass Pass : pas de chanson
Shone d'Holocost : pas de chanson
Hervé Lardic : pas de chanson
https://api.genius.com/artists/181615/songs?per_page=50&page=1
https://api.genius.com/artists/1750/songs?per_page=50&page=1
https://api.genius.com/artists/16640/songs?per_page=50&page=1
https://api.genius.com/artists/1282/songs?per_page=50&page=1
https://api.genius.com/artists/1273/songs?per_page=50&page=1
https://api.genius.com/artists/1303/songs?per_pag

	 Deen Burbigo 0.77 %
https://api.genius.com/artists/17313/songs?per_page=50&page=4
	 Demon One 3.03 %
https://api.genius.com/artists/1273/songs?per_page=50&page=4
	 JP Manova 6.45 %
	 Charly Greane 33.33 %
https://api.genius.com/artists/1282/songs?per_page=50&page=4
https://api.genius.com/artists/15855/songs?per_page=50&page=3
Niska : 119 chansons
	 Klub des Loosers 1.67 %
	 Kalash l'Afro 30.77 %
	 Vîrus 1.85 %
	 Unité 2 Feu 6.45 %
	 Bambi Cruz 13.33 %
	 Sefyu 1.22 %
	 Alibi Montana 4.65 %
	 Le Klub des 7 9.09 %
Psy 4 de la rime : 100 chansons
https://api.genius.com/artists/11499/songs?per_page=50&page=4
https://api.genius.com/artists/1684/songs?per_page=50&page=5
	 Admiral T 2.08 %
	 Orties 13.04 %
Furax : 152 chansons
	 Tandem 6.38 %
	 Daddy Lord C 7.89 %
	 Casseurs Flowters 3.57 %
https://api.genius.com/artists/14724/songs?per_page=50&page=4
	 Lionel D 50.0 %
https://api.genius.com/artists/1303/songs?per_page=50&page=4
Faf Larage : 111 chansons
	 Tandem 8.51 %
	 Tout simplement noi

	 Le Rat Luciano 4.11 %
https://api.genius.com/artists/1961/songs?per_page=50&page=2
	 Orties 43.48 %
	 Seth Gueko 2.51 %
https://api.genius.com/artists/14019/songs?per_page=50&page=3
	 JP Manova 29.03 %
	 Alibi Montana 18.6 %
	 Bambi Cruz 60.0 %
	 Admiral T 16.67 %
	 Chiens de paille 15.38 %
	 Deen Burbigo 7.69 %
	 Le Klub des 7 40.91 %
	 Raggasonic 29.03 %
	 Unité 2 Feu 29.03 %
	 Tandem 27.66 %
	 Saïan Supa Crew 13.24 %
	 Vîrus 14.81 %
	 Booba 0.57 %
	 Nikkfurie 64.29 %
	 Tout simplement noir 19.51 %
	 Odezenne 6.49 %
	 Guizmo 3.3 %
	 Kalash Criminel 4.08 %
	 Grand Corps Malade 5.61 %
	 Furax 3.95 %
	 Tandem 29.79 %
	 Faf Larage 6.31 %
	 Casseurs Flowters 16.07 %
	 Lefa 2.45 %
	 Demon One 15.15 %
Leck : 127 chansons
	 5 majeur 37.5 %
	 Sefyu 8.54 %
	 Mac Kregor 41.38 %
	 La Caution 10.94 %
	 Niska 6.72 %
KDD : 35 chansons
	 La Fouine 0.57 %
	 Mac Kregor 44.83 %
	 Niro 2.4 %
	 Lacrim 16.33 %
https://api.genius.com/artists/1961/songs?per_page=50&page=3
	 Jazzy Bazz 8.25 %
	 La Fouine 0

	 Akhenaton 0.76 %
	 Unité 2 Feu 51.61 %
	 Grand Corps Malade 12.15 %
https://api.genius.com/artists/17296/songs?per_page=50&page=2
	 Tout simplement noir 36.59 %
	 Psy 4 de la rime 13.0 %
	 Seth Gueko 6.53 %
	 Tandem 51.06 %
	 KDD 22.86 %
	 La Caution 25.0 %
	 Admiral T 37.5 %
	 Klub des Loosers 23.33 %
	 Niska 12.61 %
	 5 majeur 66.67 %
	 Le Rat Luciano 9.59 %
	 Tandem 53.19 %
	 Intouchable 7.41 %
	 113 16.35 %
	 Deen Burbigo 13.85 %
	 Demon One 27.27 %
	 Rilès 12.0 %
	 JP Manova 51.61 %
	 Daddy Lord C 47.37 %
	 Kery James 7.29 %
	 Niro 5.77 %
	 Leck 6.3 %
	 Youssoupha 5.58 %
	 Orties 86.96 %
	 La Fouine 2.56 %
	 Jazzy Bazz 15.46 %
	 Despo Rutti 6.32 %
	 Vîrus 25.93 %
	 Furax 7.89 %
	 Akhenaton 1.02 %
	 Sefyu 17.07 %
	 Alibi Montana 39.53 %
	 Faf Larage 14.41 %
	 Guizmo 7.08 %
	 Lefa 6.37 %
	 Odezenne 19.48 %
	 Booba 2.57 %
	 Chiens de paille 30.77 %
	 Saïan Supa Crew 26.47 %
	 KDD 25.71 %
	 Leck 7.09 %
	 Niska 13.45 %
	 Mac Kregor 72.41 %
Darryl Zeuja : 84 chansons
	 Raggasonic 54.8

	 Grand Corps Malade 18.69 %
	 Le Rat Luciano 15.07 %
	 JP Manova 77.42 %
	 Unité 2 Feu 70.97 %
	 Lacrim 46.94 %
	 Al Peco 75.0 %
	 Chiens de paille 44.23 %
	 Faf Larage 20.72 %
https://api.genius.com/artists/1967/songs?per_page=50&page=2
	 113 24.04 %
	 Kery James 11.98 %
	 Niska 20.17 %
	 Daddy Lord C 68.42 %
	 La Fouine 4.55 %
	 Raggasonic 77.42 %
	 La Caution 37.5 %
	 Casseurs Flowters 39.29 %
	 Alibi Montana 53.49 %
	 Kalash Criminel 22.45 %
	 Akhenaton 3.05 %
	 Demon One 40.91 %
Georgio : 148 chansons
	 Admiral T 56.25 %
	 Vîrus 38.89 %
	 Rilès 21.33 %
	 Darryl Zeuja 15.48 %
	 Furax 12.5 %
	 Odezenne 29.87 %
	 Demon One 42.42 %
	 Tout simplement noir 58.54 %
	 Youssoupha 8.58 %
	 Gringe 9.09 %
	 Klub des Loosers 36.67 %
	 Deen Burbigo 20.0 %
	 Guizmo 10.85 %
	 5 majeur 100.0 %
	 Mac Kregor 96.55 %
	 Demon One 43.94 %
	 Leck 12.6 %
	 Lefa 9.31 %
	 Ridsa 26.03 %
	 Intouchable 44.44 %
	 Despo Rutti 11.05 %
	 KDD 48.57 %
	 Saïan Supa Crew 38.24 %
	 Darryl Zeuja 16.67 %
	 Sefyu 26.83 

...

	 Mister You 93.06 %
	 Black Kent 77.14 %
	 Salif 75.97 %
	 El Matador 98.91 %
	 Grems 97.97 %
	 Mac Tyer 97.36 %
	 Nekfeu 53.85 %
	 Hayce Lemsi 82.8 %
	 Iron Sy 60.78 %
	 Jul 44.37 %
	 Maska 14.18 %
	 Jorrdee 96.68 %
	 Orelsan 87.27 %
	 Naps 44.74 %
	 Stomy Bugsy 30.77 %
	 Iron Sy 62.75 %
	 PNL 75.51 %
	 Jorrdee 97.05 %
	 Orelsan 87.88 %
	 Zesau 78.95 %
	 Naps 45.61 %
	 Lucio Bukowski 42.98 %
	 Jorrdee 97.42 %
	 Maître Gims 79.55 %
	 Black M 38.49 %
	 Maska 14.89 %
	 MZ 87.38 %
	 Sofiane 51.54 %
	 Maître Gims 79.83 %
	 Orelsan 88.48 %
	 Iron Sy 64.71 %
	 Zesau 80.7 %
	 MC Solaar 80.45 %
	 Salif 76.74 %
	 Maître Gims 80.11 %
	 Grems 98.65 %
	 Lucio Bukowski 43.4 %
	 Fabe 98.96 %
	 Nekfeu 54.12 %
	 Mister You 93.64 %
	 El Matador 100.0 %
	 Jul 44.54 %
	 Maître Gims 80.4 %
	 Stupeflip 87.02 %
	 Salif 77.52 %
	 Black M 38.91 %
	 Black Kent 77.86 %
	 Hayce Lemsi 83.33 %
	 Jorrdee 97.79 %
	 Iron Sy 66.67 %
	 Stomy Bugsy 32.69 %
	 PNL 77.55 %
	 Mister You 94.22 %
	 Nekfeu 54.4 %
	 Maître Gim

	 Nekfeu 59.62 %
	 Naps 63.16 %
	 Hayce Lemsi 91.94 %
	 Stomy Bugsy 78.85 %
	 Sofiane 62.56 %
https://api.genius.com/artists/15879/songs?per_page=50&page=1
	 Stomy Bugsy 80.77 %
	 Maska 37.59 %
	 Black Kent 90.71 %
	 Lucio Bukowski 51.49 %
	 Nekfeu 59.89 %
	 Maska 38.3 %
	 Naps 64.04 %
	 Jul 48.12 %
	 Salif 95.35 %
	 Maître Gims 89.49 %
	 Black M 48.12 %
	 Salif 96.12 %
	 Black M 48.54 %
	 Maître Gims 89.77 %
	 MC Solaar 92.18 %
https://api.genius.com/artists/15879/songs?per_page=50&page=2
	 Sofiane 63.0 %
	 Stomy Bugsy 82.69 %
	 Hayce Lemsi 92.47 %
	 Jul 48.29 %
	 Nekfeu 60.16 %
	 Black Kent 91.43 %
	 Sofiane 63.44 %
	 Maska 39.01 %
	 Naps 64.91 %
Svinkels : 68 chansons
	 Lucio Bukowski 51.91 %
	 Maska 39.72 %
	 MC Solaar 92.74 %
	 Salif 96.9 %
	 Nekfeu 60.44 %
	 Maître Gims 90.06 %
	 Black M 48.95 %
	 Stomy Bugsy 84.62 %
	 Black Kent 92.14 %
	 Jul 48.46 %
	 Black M 49.37 %
	 Hayce Lemsi 93.01 %
	 Maska 40.43 %
	 Maître Gims 90.34 %
	 Naps 65.79 %
	 Sofiane 63.88 %
	 Stomy Bugsy 86.54

	 Maska 73.76 %
	 La Rumeur 11.63 %
	 Jul 53.07 %
	 Maska 74.47 %
	 Lucio Bukowski 64.26 %
	 Svinkels 50.0 %
	 Naps 92.11 %
	 Black M 65.69 %
	 La Rumeur 12.4 %
	 Sofiane 80.18 %
	 Maître Gims 99.43 %
	 Nekfeu 69.51 %
	 Maska 75.18 %
	 La Rumeur 13.18 %
	 Sofiane 80.62 %
	 Maska 75.89 %
	 Jul 53.24 %
	 Lucio Bukowski 64.68 %
	 Svinkels 51.47 %
	 Naps 92.98 %
	 Maître Gims 99.72 %
	 Nekfeu 69.78 %
	 Svinkels 52.94 %
	 Nekfeu 70.05 %
	 Black M 66.11 %
	 Sofiane 81.06 %
	 La Rumeur 13.95 %
	 Maska 76.6 %
	 Jul 53.41 %
	 Svinkels 54.41 %
	 Black M 66.53 %
	 Naps 93.86 %
	 Lucio Bukowski 65.11 %
	 Maître Gims 100.0 %
	 Maska 77.3 %
	 Svinkels 55.88 %
	 Nekfeu 70.33 %
	 Sofiane 81.5 %
https://api.genius.com/artists/112925/songs?per_page=50&page=1
	 Jul 53.58 %
	 Maska 78.01 %
	 La Rumeur 14.73 %
	 Sofiane 81.94 %
	 Naps 94.74 %
	 Nekfeu 70.6 %
	 Svinkels 57.35 %
	 Black M 66.95 %
	 Maska 78.72 %
	 Sofiane 82.38 %
	 Sofiane 82.82 %
	 Svinkels 58.82 %
	 Black M 67.36 %
	 La Rumeur 15.5 %
	 Jul

	 Nekfeu 83.52 %
	 La Rumeur 55.04 %
	 Nekfeu 83.79 %
	 La Rumeur 55.81 %
	 Jul 61.09 %
	 Lucio Bukowski 83.4 %
	 Black M 92.47 %
	 Black M 92.89 %
	 Lucio Bukowski 83.83 %
	 La Rumeur 56.59 %
	 Nekfeu 84.07 %
	 Jul 61.26 %
	 Lucio Bukowski 84.26 %
	 La Rumeur 57.36 %
	 La Rumeur 58.14 %
	 Black M 93.31 %
	 Nekfeu 84.34 %
	 Jul 61.43 %
	 Lucio Bukowski 84.68 %
	 Black M 93.72 %
	 Black M 94.14 %
	 Jul 61.6 %
	 La Rumeur 58.91 %
	 Nekfeu 84.62 %
	 Lucio Bukowski 85.11 %
	 Jul 61.77 %
	 Black M 94.56 %
	 La Rumeur 59.69 %
	 Lucio Bukowski 85.53 %
	 La Rumeur 60.47 %
	 Nekfeu 84.89 %
	 Jul 61.95 %
	 La Rumeur 61.24 %
	 Nekfeu 85.16 %
	 Black M 94.98 %
	 Black M 95.4 %
	 Lucio Bukowski 85.96 %
	 La Rumeur 62.02 %
	 Jul 62.12 %
	 Nekfeu 85.44 %
	 La Rumeur 62.79 %
	 Nekfeu 85.71 %
	 Black M 95.82 %
	 Lucio Bukowski 86.38 %
	 Nekfeu 85.99 %
	 Jul 62.29 %
	 Black M 96.23 %
	 La Rumeur 63.57 %
	 Nekfeu 86.26 %
	 Black M 96.65 %
	 Lucio Bukowski 86.81 %
	 Jul 62.46 %
	 Black M 97.07 %
	 La Rume

	 Jul 94.03 %
	 Jul 94.2 %
	 Jul 94.37 %
	 Jul 94.54 %
	 Jul 94.71 %
	 Jul 94.88 %
	 Jul 95.05 %
	 Jul 95.22 %
	 Jul 95.39 %
	 Jul 95.56 %
	 Jul 95.73 %
	 Jul 95.9 %
	 Jul 96.08 %
	 Jul 96.25 %
	 Jul 96.42 %
	 Jul 96.59 %
	 Jul 96.76 %
	 Jul 96.93 %
	 Jul 97.1 %
	 Jul 97.27 %
	 Jul 97.44 %
	 Jul 97.61 %
	 Jul 97.78 %
	 Jul 97.95 %
	 Jul 98.12 %
	 Jul 98.29 %
	 Jul 98.46 %
	 Jul 98.63 %
	 Jul 98.81 %
	 Jul 98.98 %
	 Jul 99.15 %
	 Jul 99.32 %
	 Jul 99.49 %
	 Jul 99.66 %
	 Jul 99.83 %
	 Jul 100.0 %
https://api.genius.com/artists/11491/songs?per_page=50&page=1
Expression Direkt : 35 chansons
	 Expression Direkt 2.86 %
	 Expression Direkt 5.71 %
	 Expression Direkt 8.57 %
	 Expression Direkt 11.43 %
	 Expression Direkt 14.29 %
	 Expression Direkt 17.14 %
	 Expression Direkt 20.0 %
	 Expression Direkt 22.86 %
	 Expression Direkt 25.71 %
	 Expression Direkt 28.57 %
	 Expression Direkt 31.43 %
	 Expression Direkt 34.29 %
	 Expression Direkt 37.14 %
	 Expression Direkt 40.0 %
	 Expression Direk